In [17]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects import numpy2ri

import pandas as pd
import numpy as np

pandas2ri.activate()

In [19]:
np.random.seed(1)

# environment 1
na = 140
X1a = 0.3 * np.random.randn(na)
X3a = X1a + 0.2 * np.random.randn(na)
Ya = -.7 * X1a + .6 * X3a + 0.1 * np.random.randn(na)
X2a = -0.5 * Ya + 0.5 * X3a + 0.1 * np.random.randn(na)

# environment 2
nb = 80
X1b = 0.3 * np.random.randn(nb)
X3b = 0.5 * np.random.randn(nb)
Yb = -.7 * X1b + .6 * X3b + 0.1 * np.random.randn(nb)
X2b = -0.5 * Yb + 0.5 * X3b + 0.1 * np.random.randn(nb)

# Combine environments
X1 = np.concatenate((X1a, X1b))
X2 = np.concatenate((X2a, X2b))
X3 = np.concatenate((X3a, X3b))
Y = np.concatenate((Ya, Yb))

Xmatrix = np.column_stack((X1, X2, X3))

# Convert numpy arrays to R objects
r_Xmatrix = numpy2ri.numpy2ri(Xmatrix)
r_Y = numpy2ri.numpy2ri(Y)

# Load seqICP library
robjects.r('''
    library(seqICP)
''')

# Run the seqICP analysis
robjects.globalenv['Xmatrix'] = r_Xmatrix
robjects.globalenv['Y'] = r_Y
robjects.r('''
    seqICP_result <- seqICP(X = Xmatrix, Y = Y, 
        par.test = list(grid = seq(0, length(Y), length(Y)/10), complements = FALSE, link = sum,
        alpha = 0.05, B = 100), max.parents = 4, stopIfEmpty = FALSE, silent = TRUE)
    seqICP_summary <- summary(seqICP_result)
''')

# Retrieve the results
seqICP_summary = robjects.r['seqICP_summary']


 Invariant Linear Causal Regression at level 0.05

 Variables X1, X3 show a significant causal effect

 
 
         
 coefficient
 lower bound
 upper bound
  p-value
  

intercept
       -0.01
     -0.0430
      0.0286
       NA
  

X1       
       -0.73
     -0.7870
     -0.5064
     0.02
 *

X2       
        0.00
      0.0000
      0.0000
     0.42
  

X3       
        0.60
      0.5600
      0.7210
     0.02
 *


---
Signif. codes:  

0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1








In [20]:

# Retrieve the results
lm_result = robjects.r['lm_result']

# Print the results
print(lm_result)


Call:
lm(formula = Y ~ Xmatrix)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.205831 -0.061317 -0.001113  0.057515  0.266640 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.001799   0.005980   0.301    0.764    
XmatrixX1   -0.583158   0.027397 -21.285  < 2e-16 ***
XmatrixX2   -0.379482   0.047765  -7.945 1.06e-13 ***
XmatrixX3    0.687121   0.018082  38.000  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.08813 on 216 degrees of freedom
Multiple R-squared:  0.904,	Adjusted R-squared:  0.9027 
F-statistic: 678.1 on 3 and 216 DF,  p-value: < 2.2e-16




In [21]:
robjects.r['parent_set']

array([1, 3], dtype=int32)